### 27. ¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la plataforma? (⭐⭐)

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [3]:
dfLogs = pd.read_csv("../Dataset/logs.csv",
                     usecols = ["timestamp", "logtype", "action", "title"],
                     dtype = {"logtype": "category", "action": "category"},
                     parse_dates = ["timestamp"])
dfLogs.head()

,timestamp,logtype,action,title
0,2004-12-23 07:10:09+00:00,delete,delete,Verificacionismo
1,2004-12-23 07:14:09+00:00,delete,delete,Rapunzel
2,2004-12-23 07:26:10+00:00,delete,delete,Macromedia Flash
3,2004-12-23 07:55:09+00:00,delete,delete,Archivo:Gnu head.png
4,2004-12-23 08:02:22+00:00,delete,delete,Archivo:Gnu-head-sm.jpg


In [4]:
dfBlock = dfLogs.loc[(dfLogs["logtype"] == "block") & (dfLogs["action"] == "block")]
dfBlock = dfBlock.groupby("title").agg({"timestamp": "min"}).reset_index() # Me quedo con el primer bloqueo
dfBlock.head()

,title,timestamp
0,Usuario:! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,2013-03-24 07:12:49+00:00
1,Usuario:!! !!ПОМОГИТЕ СЕСТРЕ НА ОПЕРАЦИЮ!! !!,2017-11-16 17:22:09+00:00
2,"Usuario:!""·$%&",2012-01-04 02:51:53+00:00
3,Usuario:!23455666A!wee,2020-05-29 09:34:26+00:00
4,Usuario:!2o%Uv6PHBj6JfgYmfgqVx!SCEBEP4,2020-05-13 17:40:04+00:00


In [5]:
dfNewUser = dfLogs[(dfLogs["logtype"] == "newusers") & (dfLogs["action"] == "create")]
dfNewUser = dfNewUser.groupby("title").agg({"timestamp": "min"}).reset_index() # Me quedo con el primer registro
dfNewUser.head()

,title,timestamp
0,Usuario:! DanSkammelsrod !,2006-12-28 17:07:56+00:00
1,Usuario:! Mostaza 79,2021-05-05 21:53:19+00:00
2,Usuario:! Soul07089,2021-08-26 12:25:26+00:00
3,Usuario:! x0ffDeseadaah,2021-03-15 00:44:03+00:00
4,Usuario:!! juan !! wuanchope !!,2016-03-17 02:42:47+00:00


In [6]:
dfMerged = dfNewUser[["title", "timestamp"]].merge(dfBlock[["timestamp", "title"]],
                                                   on = "title",
                                                   suffixes = ("_create", "_block"))
dfMerged.head()

,title,timestamp_create,timestamp_block
0,"Usuario:!""·$%&",2008-08-21 22:56:10+00:00,2012-01-04 02:51:53+00:00
1,Usuario:!Maritzadechile!,2012-06-27 20:56:20+00:00,2012-08-04 14:43:47+00:00
2,Usuario:!Noticias K-pop Informa!,2020-12-16 00:54:45+00:00,2021-03-01 01:31:13+00:00
3,Usuario:!asesinoflow!!,2013-10-26 17:18:27+00:00,2019-10-06 15:19:58+00:00
4,Usuario:!emprendiciencia innovación social,2017-07-28 09:59:43+00:00,2020-05-26 14:36:42+00:00


In [7]:
dfMerged["diferencia"] = dfMerged["timestamp_block"] - dfMerged["timestamp_create"]
dfMerged.head()

,title,timestamp_create,timestamp_block,diferencia
0,"Usuario:!""·$%&",2008-08-21 22:56:10+00:00,2012-01-04 02:51:53+00:00,1230 days 03:55:43
1,Usuario:!Maritzadechile!,2012-06-27 20:56:20+00:00,2012-08-04 14:43:47+00:00,37 days 17:47:27
2,Usuario:!Noticias K-pop Informa!,2020-12-16 00:54:45+00:00,2021-03-01 01:31:13+00:00,75 days 00:36:28
3,Usuario:!asesinoflow!!,2013-10-26 17:18:27+00:00,2019-10-06 15:19:58+00:00,2170 days 22:01:31
4,Usuario:!emprendiciencia innovación social,2017-07-28 09:59:43+00:00,2020-05-26 14:36:42+00:00,1033 days 04:36:59


In [8]:
# Hay incoherencias donde se bloquea a un usuario antes de que se registre.
# Se omiten esos valores
dfMerged[dfMerged["diferencia"] >= timedelta(0)].nsmallest(1, "diferencia").iloc[0].diferencia

Timedelta('0 days 00:00:04')